In [ ]:
import numpy as np, random, operator, pandas as pd, matplotlib.pyplot as plt
from math import dist
from CSVReader import CSVReader
from DR import DR
from SK import SK
from UAV import UAV
from GraphInfo import GraphInfo
from Node import Node
from Path import Path

class Fitness:    #각 개체에 대한 적합도를 관리

    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness= 0.0

    def get_routeDistance(self):    #거리 매기기
        if self.distance ==0:
            pathDistance = 0
            depot_node = tau_graph.get_node(0)
            pathDistance += depot_node.get_distance(self.route[0])
            for i in range(0, len(self.route)-1):
                node = tau_graph.get_node(self.route[i])    #현 노드 기준으로
                pathDistance += node.get_distance(self.route[i+1])     #다음 노드까지 거리 적산
            pathDistance += depot_node.get_distance(self.route[-1])
            self.distance = pathDistance
        
        return self.distance
            
    def routeFitness(self):    #거리 값의 역순(작아야 좋으므로) 으로 점수 매기기
        if self.fitness == 0:
            self.fitness = 1 / float(self.get_routeDistance())
        return self.fitness



def createRoute(nodelist):     #random하게 sample route생성
    route = random.sample(nodelist, len(nodelist))     
    return route


def initialPopulation(popSize, nodelist):     #초기 population 선정, pop size 100개, nodeList 입력
    population = []
    for i in range(0, popSize):
        population.append(createRoute(nodelist))     #nodelist(20개)를 100개로 재생성 함(증폭)
    return population


def rankRoutes(population):     #population 100개에 대해서 순위를 매김
    fitnessResults = {}     #dict.
    for i in range(0,len(population)):
        fitnessResults[i] = Fitness(population[i]).routeFitness()     #dict.를 routeFitness로 채움, {i:routeFitness점수}
    #print(fitnessResults)
    #a = sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)
    #print(a)
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)     # 점수가 높은 순서대로 정렬


def selection(popRanked, eliteSize):     #가중 비례, 토너먼트, 랭크 처리된 pop을 입력
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])     #행렬 만들어서 룰렛 돌리기
    df['cum_sum'] = df.Fitness.cumsum()     #누적 합
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()     #각각에 대해 가중치 반영(%)
    for i in range(0, eliteSize):     #선정된 엘리트 수만큼 반복
        selectionResults.append(popRanked[i][0])     #노드 path를 순서대로 넣는다
    for i in range(0, len(popRanked) - eliteSize):     #가장 좋은 값 반환(선택)
        pick = 100*random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i,3]:     #패스트 인덱싱
                selectionResults.append(popRanked[i][0])
                break
    return selectionResults


def matingPool(population, selectionResults):     #셀렉션 된 개체(리스트)들 정리하여 mating pool에 넣기
    matingpool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool


def breed(parent1, parent2):     #번식, 노드 순서중에 랜덤하게 발췌하여 뒤집음
    child = []
    childP1 = []
    childP2 = []
    
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))
    
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):     #특정 노드 구간만 랜덤으로 찢어서 랜덤 돌려서 교체
        childP1.append(parent1[i])
        
    childP2 = [item for item in parent2 if item not in childP1]     #crossover!!!

    child = childP1 + childP2
    return child

def breedPopulation(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0,eliteSize):
        children.append(matingpool[i])
    
    for i in range(0, length):
        child = breed(pool[i], pool[len(matingpool)-i-1])     #선정된 pop size 만큼 breed 시행
        children.append(child)
    return children


def mutate(individual, mutationRate):     #변이, 로컬 벗어나기 위한 과정
    for swapped in range(len(individual)):     #노드끼리 직접 swap 한다
        if(random.random() < mutationRate):
            swapWith = int(random.random() * len(individual))
            
            node1 = individual[swapped]
            node2 = individual[swapWith]
            
            individual[swapped] = node2
            individual[swapWith] = node1
    return individual

def mutatePopulation(population, mutationRate):
    mutatedPop = []
    
    for ind in range(0, len(population)):
        mutatedInd = mutate(population[ind], mutationRate)
        mutatedPop.append(mutatedInd)
    return mutatedPop


#세대 승계
def nextGeneration(currentGen, eliteSize, mutationRate):     #입력 받아서 다음 세대 반복(함수 이어주기)
    popRanked = rankRoutes(currentGen)
    selectionResults = selection(popRanked, eliteSize)
    matingpool = matingPool(currentGen, selectionResults)
    children = breedPopulation(matingpool, eliteSize)
    nextGeneration = mutatePopulation(children, mutationRate)
    return nextGeneration


def geneticAlgorithm(population, popSize, eliteSize, mutationRate, generations):
    pop = initialPopulation(popSize, population)     #population 리스트 100개 생성
    print("Initial distance: " + str(1 / rankRoutes(pop)[0][1]))     #가장 첫번째 poputation 개체의 두번째 원소, 즉 dist.의 역순을 취함
    
    for i in range(0, generations):    #세대수 만큼 돌아가면서,
        pop = nextGeneration(pop, eliteSize, mutationRate)     #다음 세대 연산(pop 리스트와, 엘리트 사ㅣ즈와 와 변이율 필요), pop 리스트 안에서 모든 변이가 이루어짐
    
    print("Final distance: " + str(1 / rankRoutes(pop)[0][1]))     #가장 앞에 위치하게 되는(가장 fitness 점수가 좋은) 개체의 두번째 원소, 즉 dist.의 역순을 취함
    bestRouteIndex = rankRoutes(pop)[0][0]     #가장 첫번째 poputation 개체의 첫번째 원소, 즉 노드 순서의 인덱스
    bestRoute = pop[bestRouteIndex]     #해당 인덱스의 값
    return bestRoute



: 

In [59]:
import numpy as np, random, operator, pandas as pd, matplotlib.pyplot as plt
from math import dist
from CSVReader import CSVReader
from DR import DR
from SK import SK
from UAV import UAV
from GraphInfo import GraphInfo
from Node import Node
from Path import Path


reader = CSVReader() #CSV 파일 읽어옴
tau_graph = reader.load_csv('tau_v2.csv')
taup_graph = reader.load_csv('taup_v2.csv')
tau_graph.init_nearest_nodes()  #nearest 노드로 정렬 초기화
taup_graph.init_nearest_nodes()

truck_nodes_list = [node.node_num for node in tau_graph.nodes]     #노드 번호 리스트
truck_nodes_list.remove(0)
print("truck_nodes_list", truck_nodes_list)


temp = initialPopulation(10, truck_nodes_list)
print(temp)

temp_2 = rankRoutes(temp)



geneticAlgorithm(population=truck_nodes_list, popSize=100, eliteSize=15, mutationRate=0.01, generations=500)


truck_nodes_list [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
[[2, 11, 7, 14, 6, 9, 1, 13, 18, 3, 16, 17, 10, 4, 15, 19, 12, 8, 20, 5], [17, 1, 3, 18, 6, 19, 8, 15, 20, 14, 10, 13, 5, 4, 16, 2, 9, 11, 12, 7], [2, 17, 4, 10, 16, 1, 13, 15, 11, 6, 7, 5, 20, 12, 18, 3, 9, 19, 8, 14], [2, 12, 5, 15, 11, 13, 20, 10, 9, 7, 3, 14, 17, 16, 8, 18, 19, 6, 4, 1], [7, 2, 1, 17, 4, 16, 3, 14, 9, 15, 20, 10, 5, 13, 8, 6, 11, 12, 18, 19], [11, 4, 1, 18, 15, 17, 5, 8, 14, 19, 7, 16, 10, 12, 3, 9, 13, 6, 20, 2], [12, 7, 15, 11, 20, 10, 19, 8, 14, 13, 4, 1, 5, 18, 3, 9, 6, 17, 16, 2], [10, 9, 13, 20, 7, 11, 12, 18, 14, 1, 19, 5, 16, 6, 15, 8, 3, 17, 2, 4], [9, 20, 19, 5, 13, 16, 17, 14, 8, 12, 1, 15, 7, 4, 2, 10, 11, 3, 18, 6], [4, 1, 11, 15, 16, 5, 7, 17, 14, 9, 3, 13, 18, 20, 6, 2, 19, 12, 10, 8]]
Initial distance: 4655.0
Final distance: 2383.0


[1, 14, 15, 6, 4, 9, 3, 10, 18, 8, 17, 5, 7, 12, 11, 19, 16, 2, 13, 20]

In [34]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
a.append(0)
a.insert(0,0)
a

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 0]